In [77]:
import pandas as pd
permits = pd.read_csv('msaannual_201999.xlsx - MSA Units.csv')

In [78]:
import bamboolib

In [79]:
split_df = permits['Name'].str.split(',', expand=True)
split_df.columns = ['Name' + f"_{id_}" for id_ in range(len(split_df.columns))]
permits = pd.merge(permits, split_df, how="left", left_index=True, right_index=True)
cleaned_column_names = (permits.columns
                        .str.strip()
                        .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                        .str.lower()
                        .str.replace('[ _-]+', '_')
                        .str.replace('[}{)(><.!?\\\\:;,-]', ''))
permits.columns = cleaned_column_names

In [80]:
permits.rename({'name_1': 'state'}, axis=1, inplace=True)
permits.head()

,csa,cbsa,name,total,1_unit,2_units,3_and_4_units,5_units_or_more,num_of_structures_with_5_units_or_more,name_0,state
0,999,10180,"Abilene, TX ...",370,354,16,0,0,0,Abilene,TX ...
1,184,10420,"Akron, OH ...",945,856,2,32,55,1,Akron,OH ...
2,999,10500,"Albany, GA ...",402,182,0,0,220,10,Albany,GA ...
3,440,10540,"Albany-Lebanon, OR ...",716,466,0,0,250,26,Albany-Lebanon,OR
4,104,10580,"Albany-Schenectady-Troy, NY ...",1870,1120,40,45,665,48,Albany-Schenectady-Troy,NY


In [81]:
permits.rename({'name_o': 'area'}, axis=1, inplace=True)
permits = permits.drop(columns=['name'])

In [82]:
pop = pd.read_csv('cbsa-met-Population Estimate.xlsx - CBSA-MET-EST2019-ANNRES.csv')

In [83]:
header_row = 0
pop.columns = pop.iloc[header_row]
pop = pop.loc[~(pop['area'].str.contains('area', case=True, regex=False, na=False))]
split_df = pop['area'].str.split(',', n=1, expand=True)
split_df.columns = ['area' + f"_{id_}" for id_ in range(len(split_df.columns))]
pop = pd.merge(pop, split_df, how="left", left_index=True, right_index=True)

In [84]:
pop = pop.drop(columns = ['area'])

In [85]:
pop.rename({'area_1': 'state'}, axis=1, inplace=True)
pop.rename({'area_0': 'area'}, axis=1, inplace=True)

In [88]:
pop['area'] = pop['area'].map(lambda x: x.lstrip('.'))

In [87]:
pop_permits = pd.merge(pop, permits, how='inner', left_on=['area'], right_on=['name_0'])
pop_permits = pop_permits.drop(columns=['state_x'])
pop_permits = pop_permits.loc[pop_permits['state_y'].str.contains('CA', case=True, regex=False, na=False)]
pop_permits = pop_permits.replace(',','', regex=True)
pop_permits['2019'] = pd.to_numeric(pop_permits['2019'], downcast='integer', errors='coerce')
pop_permits['2018'] = pd.to_numeric(pop_permits['2018'], downcast='integer', errors='coerce')
pop_permits

,2020,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,csa,cbsa,total,1_unit,2_units,3_and_4_units,5_units_or_more,num_of_structures_with_5_units_or_more,name_0,state_y
22,839621,840996,847970,853606,862000,869176,876031,880856,887356,893758,...,999,12540,2261,2190,12,24,35,6,Bakersfield,CA
71,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,...,999,17020,1426,885,6,64,471,36,Chico,CA ...
116,174524,174716,175786,176575,176438,177974,178937,180179,181649,181523,...,999,20940,398,266,0,0,132,11,El Centro,CA ...
138,930507,932039,939406,945045,951514,960567,969488,976830,985238,991950,...,260,23420,3393,2441,60,213,679,76,Fresno,CA ...
161,152974,152370,151868,150991,150337,149495,150085,149382,149665,151382,...,260,25260,409,382,2,0,25,3,Hanford-Corcoran,CA
233,12828957,12838417,12925753,13013443,13097434,13166609,13234696,13270694,13278000,13249879,...,348,31080,30554,9306,1694,358,19196,516,Los Angeles-Long Beach-Anaheim,CA
236,150834,150986,151675,151527,151370,153456,153576,153956,155423,156882,...,260,31460,644,644,0,0,0,0,Madera,CA ...
245,255796,256721,259297,260867,262026,264419,266353,267628,271096,274151,...,488,32900,948,948,0,0,0,0,Merced,CA ...
256,514450,515145,517560,520424,523451,528015,533211,539255,544717,548126,...,488,33700,699,611,4,3,81,7,Modesto,CA ...
268,136535,136759,137691,138538,139580,140405,140862,140840,139878,138789,...,488,34900,749,135,2,4,608,1,Napa,CA ...


In [93]:
pop_permits['growth_19'] = pop_permits['2019'] - pop_permits['2018']

In [90]:
pop_permits['suply_demand19'] = pop_permits['total']/pop_permits['growth_19']

In [97]:
pop_permits = pop_permits.sort_values(by=['suply_demand19'], ascending=[True])
import plotly.express as px
fig = px.bar(pop_permits, x='area', y='suply_demand19')
fig